# Transform data


Transforms the datasets we have to .xyz format, so they are compatible with the CryinGAN repository.

In [ ]:
import os
import sys

from pathlib import Path

%cd ..

from src.utils import load_raw_data, read_raw_sample

%cd -

def makedir_if_not_exists(path):
    if not os.path.isdir(path):
        print("Creating directory {}".format(path))
        os.mkdir(path)
    return

path = Path("../data/raw/samples")


phis = [0.84]
files, dataframe, metadata = load_raw_data(path, phi=phis)


In [ ]:
file

In [ ]:
Path(file.replace("raw", "processed")).parent

In [ ]:
import pandas as pd

max_files = 50 # Limit the amount of data

processed_folder = Path("../data/processed/samples")

makedir_if_not_exists(processed_folder)

# Dump as xyz with all data in a single file
processed_file = Path(files[0].replace("raw", "processed")).parent

makedir_if_not_exists(str(processed_file.parent))


os.remove(processed_file)

with open(processed_file, "a+") as f:

    for i, file in enumerate(files):
        dataframe, metadata = read_raw_sample(file)

        xyz = dataframe[["class", "x", "y"]].reset_index(drop=True)

        N = metadata.iloc[0,0] # N particles

        xyz["z"] = 0

        xyz["class2"] = xyz["class"] # NOTE: This saves the files
        xyz["class"] = "Cu" # TODO: change to real atom symbols

        f.write(f"{N}\n")
        # s = 'Properties=species:S:1:pos:R:3 pbc="F F F"' # for extxyz format – unsure if correct
        # f.write(s) # NOTE: Write this for extxyz format
        f.write("\n")
        f.flush()

        xyz.to_csv(
            processed_file,
            header=False,
            index=False, 
            sep="\t", 
            mode="a+",
            
            )
        
        if i > max_files:
            break
        



display(dataframe.head(10))
metadata

In [ ]:
from ase.io import read
slab_from_file = read(processed_file, index=0, format="xyz")

In [ ]:
from ase.visualize import view
view(slab_from_file)

Works